<a href="https://colab.research.google.com/github/rubuntu/Taller_Introduccion_a_Ciencia_de_Datos_IA_e_Ingenieria_de_Datos/blob/main/sesion_12_clasificacion_de_perros_y_gatos_con_resnet18_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación de Perros y Gatos con ResNet18 (Transfer Learning)

## Objetivos
- Aprender a cargar datasets de imágenes desde HuggingFace (`microsoft/cats_vs_dogs`).
- Preparar un pipeline de preprocesamiento con **transformaciones y data augmentation**.
- Usar un modelo preentrenado (**ResNet18 con pesos de ImageNet**) y adaptarlo al problema de clasificación binaria.
- Practicar el concepto de **congelar capas y entrenar solo las últimas** (fine-tuning parcial).
- Implementar **entrenamiento con early stopping** para evitar overfitting.
- Evaluar el modelo y visualizar ejemplos de predicciones correctas e incorrectas.

# Tipos de Redes Neuronales

---

## 🌐 1. Redes Clásicas (feedforward)

* **Perceptrón**: la más simple, una sola capa de neuronas.
* **Perceptrón Multicapa (MLP / FFNN)**: varias capas densas conectadas hacia adelante, sin ciclos. Muy usadas en datos tabulares, predicciones simples y como “bloques básicos” de arquitecturas más complejas.

---

## 📸 2. Redes Convolucionales (CNN)

* Diseñadas para datos con estructura espacial (imágenes, video, audio).
* Usan **filtros** para detectar patrones locales (bordes, texturas).
* Variantes:

  * **LeNet** (histórica, dígitos MNIST).
  * **AlexNet, VGG, ResNet, EfficientNet** (vision moderna).
  * **Conv1D/Conv2D/Conv3D** para señales, imágenes o volúmenes.
  * **U-Net / SegNet** para segmentación (ej. en medicina).

---

## 🕰 3. Redes Recurrentes (RNN)

* Para datos **secuenciales** (texto, series de tiempo, audio).
* La salida depende del estado previo.
* Variantes:

  * **RNN simple** (difíciles de entrenar).
  * **LSTM (Long Short-Term Memory)**: maneja dependencias largas.
  * **GRU (Gated Recurrent Unit)**: más liviana que LSTM.

---

## 🧭 4. Redes Basadas en Atención y Transformers

* Superaron a las RNN en NLP.
* Mecanismo clave: **self-attention**, que permite ver relaciones globales en una secuencia.
* Usadas en:

  * **NLP** (BERT, GPT, T5).
  * **Visión** (Vision Transformers - ViT).
  * **Multimodalidad** (CLIP, Flamingo).
  * **Modelos generativos** (Stable Diffusion, Llama, ChatGPT).

---

## 🎨 5. Redes Generativas

* Aprenden a **crear datos nuevos** similares a los de entrenamiento.
* Principales tipos:

  * **Autoencoders (AE, VAE)**: comprimen y reconstruyen datos.
  * **GANs (Generative Adversarial Networks)**: generador vs discriminador.
  * **Flow-based models**: transformaciones invertibles (Normalizing Flows).
  * **Diffusion Models**: modelos de difusión (hoy dominan en imágenes).

---

## 🧩 6. Redes Especializadas

* **Redes de Hopfield**: memoria asociativa.
* **SOM (Self-Organizing Maps)**: mapas auto-organizados para clustering.
* **Boltzmann Machines** y **Restricted Boltzmann Machines (RBM)**: usadas en preentrenamiento.
* **Capsule Networks**: intentan superar limitaciones de las CNN.
* **Graph Neural Networks (GNNs)**: para datos en forma de grafo (redes sociales, química, logística).

---

👉 Como ves, el **tipo de red** depende mucho del **tipo de dato** y del **objetivo**:

* Tabular → MLP
* Imágenes → CNN / ViT
* Texto / Secuencias → RNN / Transformers
* Generación → AE / GAN / Diffusion
* Datos relacionales → GNN

---



## 📊 Comparativo de Tipos de Redes Neuronales

| Tipo de Red                     | Ventajas                                                                       | Desventajas                                                                              | Casos de Uso                                                    |
| ------------------------------- | ------------------------------------------------------------------------------ | ---------------------------------------------------------------------------------------- | --------------------------------------------------------------- |
| **MLP (Perceptrón Multicapa)**  | Sencilla de implementar, funciona bien en datos tabulares, sirve como baseline | No escala bien con imágenes o secuencias, poca capacidad para captar estructura compleja | Scoring de crédito, predicciones tabulares, forecasting simple  |
| **CNN (Convolucionales)**       | Excelentes en visión, detectan patrones locales, eficientes en imágenes        | Requieren muchos datos, sensibles a transformaciones (rotación, escala)                  | Clasificación de imágenes, visión médica, vehículos autónomos   |
| **RNN (Recurrentes)**           | Manejan secuencias, modelan dependencias temporales                            | Problemas de gradiente (exploding/vanishing), entrenamiento lento                        | Series de tiempo, análisis de texto, speech recognition         |
| **LSTM / GRU**                  | Capturan dependencias largas, más estables que RNN clásicas                    | Computacionalmente costosas, menos usadas hoy frente a Transformers                      | Traducción automática, predicción de secuencias, chatbots       |
| **Transformers**                | Escalan muy bien, capturan relaciones globales, dominan NLP y multimodal       | Altísimo costo computacional, requieren muchos datos                                     | ChatGPT, BERT, traducción, visión con ViT, modelos multimodales |
| **Autoencoders / VAE**          | Útiles para reducción de dimensionalidad, generación controlada                | Reconstrucciones a veces borrosas, menos expresivos que GAN/Diffusion                    | Detección de anomalías, compresión, generación básica           |
| **GANs**                        | Generan datos realistas (imágenes, audio, video)                               | Entrenamiento inestable, difícil de balancear generador y discriminador                  | Deepfakes, arte digital, síntesis de imágenes                   |
| **Diffusion Models**            | Estado del arte en generación de imágenes/audio, más estables que GANs         | Lentitud en inferencia (aunque existen aceleraciones)                                    | Stable Diffusion, MidJourney, generación de música e imágenes   |
| **GNN (Graph Neural Networks)** | Capturan relaciones complejas en grafos, muy útiles en dominios estructurados  | Implementación más compleja, requieren conocimiento especializado                        | Redes sociales, química, logística, detección de fraude         |

---

👉 En resumen:

* **MLP** = tabular simple.
* **CNN** = visión.
* **RNN/LSTM/GRU** = secuencias tradicionales.
* **Transformers** = el rey actual (texto, imagen, multimodal).
* **GAN/Diffusion** = generación creativa.
* **GNN** = datos relacionales.

---

## 🌳 Árbol de Decisión: ¿Qué red usar?

```
¿Con qué tipo de datos trabajas?
│
├── Tabulares (CSV, tablas, features numéricos/categóricos)
│     └── MLP (Perceptrón Multicapa)
│
├── Imágenes
│     ├── Clasificación / Detección / Segmentación
│     │      ├── Pocos datos → CNN pre-entrenada (ResNet, EfficientNet, U-Net)
│     │      └── Muchos datos → Vision Transformer (ViT)
│     └── Generación de imágenes
│            ├── Realismo → GAN
│            └── Estado del arte → Diffusion Models
│
├── Texto o Secuencias (NLP, series de tiempo, audio)
│     ├── Dependencias cortas → RNN
│     ├── Dependencias largas → LSTM / GRU
│     └── NLP moderno → Transformer (BERT, GPT)
│
├── Datos Relacionales (Grafos, redes sociales, química)
│     └── Graph Neural Networks (GNNs)
│
└── Otros casos especiales
      ├── Reducción de dimensionalidad / Anomalías → Autoencoder / VAE
      ├── Memoria asociativa → Hopfield
      └── Organización no supervisada → Self-Organizing Maps (SOM)
```

---

## 🚀 Lectura rápida

* **Si tienes TABLAS → MLP.**
* **Si son IMÁGENES → CNN o Transformer.**
* **Si es TEXTO o SERIES → Transformer (o LSTM si es más clásico).**
* **Si son GRAFOS → GNN.**
* **Si se busca GENERAR → GAN o Diffusion.**

---


## 📌 **Data Augmentation**

* Es una técnica para **aumentar artificialmente la cantidad de datos de entrenamiento** sin recolectar nuevos ejemplos.
* Consiste en aplicar **transformaciones aleatorias** a las imágenes originales (o datos en general).
* Ejemplos comunes en imágenes:

  * Rotaciones, volteos (*flips*).
  * Cambios de brillo, contraste, color.
  * Recortes (*cropping*), escalados, zooms.
  * Adición de ruido.
* ✅ Beneficio: evita **overfitting** y mejora la capacidad de generalización del modelo.

---

## 📌 **Transfer Learning**

* Estrategia en la que un modelo previamente **entrenado en una tarea grande** (ej. clasificación en ImageNet con millones de imágenes) se reutiliza para otra tarea similar.
* Se puede:

  1. **Congelar capas** iniciales (que extraen características generales como bordes, texturas).
  2. **Reentrenar solo las capas finales** para la nueva tarea (ej. clasificar radiografías en lugar de gatos/perros).
* ✅ Beneficio: permite entrenar modelos con **pocos datos** y en **menos tiempo**.

---

## 📌 **ResNet18**

* Es una **Red Residual (Residual Network)** propuesta por Microsoft en 2015.
* "18" significa que tiene **18 capas de profundidad** (convolucionales + fully connected).
* Introduce el concepto de **skip connections (conexiones residuales)**:

  * En lugar de pasar siempre "capa → capa → capa", se permite que la entrada se sume directamente a la salida de una capa posterior.
  * Esto resuelve el problema del **desvanecimiento del gradiente** y permite entrenar redes **muy profundas** (cientos de capas).

📌 Estructura simplificada de ResNet18:

* 1 capa convolucional inicial.
* 4 bloques residuales principales, cada uno con 2 capas convolucionales.
* Una capa *fully connected* final.

---

La **ResNet18 preentrenada** que se encuentra en librerías como **PyTorch** o **Torchvision** normalmente está entrenada en **ImageNet** 🖼️, un dataset enorme con **más de 1 millón de imágenes en 1000 clases diferentes**.

👉 Dentro de esas 1000 clases se tienen categorías de:

* Animales (aves, peces, insectos, mamíferos, etc.).
* Objetos cotidianos (sillas, tazas, autos, relojes, etc.).
* Escenas naturales.

---

✅ **Entonces:**

* Gracias a **transfer learning**, se puede reutilizar y **adaptar para clasificar perros vs gatos** (binaria), ajustando solo las últimas capas.

---


### **Ejemplo sencillo en PyTorch** usando una **LSTM** para clasificación de texto (positivo/negativo) con un dataset de juguete.


### Explicación rápida:

* **Embedding**: convierte índices de palabras en vectores densos.
* **LSTM**: procesa la secuencia y devuelve un hidden state final.
* **Linear + Softmax**: convierte el hidden state en probabilidades de clases.


---


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# -----------------------------
# 1. Datos de ejemplo (toy)
# -----------------------------
# Supongamos vocabulario reducido: {0:PAD, 1:good, 2:bad, 3:movie, 4:boring, 5:great}
sentences = [
    [1, 3, 5],   # "good movie great" (positivo)
    [2, 3, 4],   # "bad movie boring" (negativo)
    [1, 5],      # "good great" (positivo)
    [2, 4]       # "bad boring" (negativo)
]
labels = [1, 0, 1, 0]  # 1: positivo, 0: negativo

# Padding para igualar longitudes
max_len = max(len(s) for s in sentences)
X = [s + [0]*(max_len-len(s)) for s in sentences]
X = torch.tensor(X)
y = torch.tensor(labels)

# -----------------------------
# 2. Definir modelo LSTM
# -----------------------------
class SentimentRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)              # (batch, seq_len, embed_dim)
        output, (h_n, c_n) = self.lstm(embedded) # h_n: (1, batch, hidden_dim)
        out = self.fc(h_n[-1])                   # Usamos el último hidden state
        return out

# -----------------------------
# 3. Entrenamiento
# -----------------------------
vocab_size = 6   # tokens del 0 al 5
embed_dim = 8
hidden_dim = 16
output_dim = 2   # positivo / negativo

model = SentimentRNN(vocab_size, embed_dim, hidden_dim, output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(20):
    optimizer.zero_grad()
    y_pred = model(X)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 5 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# -----------------------------
# 4. Prueba con nueva frase
# -----------------------------
test = torch.tensor([[1, 3, 5]])  # "good movie great"
pred = model(test)
print("Predicción:", torch.argmax(pred, dim=1).item())  # 1 = positivo

Este script crea y entrena una red neuronal simple para **análisis de sentimientos**. El objetivo es clasificar frases como positivas o negativas.

-----

### \#\# 1. Datos de Ejemplo y Preprocesamiento

En esta sección, se preparan los datos para que el modelo pueda procesarlos.

```python
# Supongamos vocabulario reducido: {0:PAD, 1:good, 2:bad, 3:movie, 4:boring, 5:great}
sentences = [
    [1, 3, 5],   # "good movie great" (positivo)
    [2, 3, 4],   # "bad movie boring" (negativo)
    [1, 5],      # "good great" (positivo)
    [2, 4]       # "bad boring" (negativo)
]
labels = [1, 0, 1, 0]  # 1: positivo, 0: negativo

# Padding para igualar longitudes
max_len = max(len(s) for s in sentences)
X = [s + [0]*(max_len-len(s)) for s in sentences]

# Conversión a tensores de PyTorch
X = torch.tensor(X)
y = torch.tensor(labels)
```

1.  **Datos Crudos**: `sentences` contiene las oraciones, pero en lugar de palabras, usa números (IDs o "tokens") que las representan. `labels` contiene la clasificación de cada oración (1 para positiva, 0 para negativa).

2.  **Padding (Relleno)**: Las redes neuronales necesitan que todas las entradas de un lote tengan el mismo tamaño. Como las oraciones `"good great"` (2 palabras) y `"good movie great"` (3 palabras) tienen longitudes distintas, se aplica un **padding**. Se añade el token `0` (que definimos como `PAD` o relleno) a las oraciones más cortas hasta que todas midan lo mismo que la más larga (que es 3).

      * `[1, 5]` se convierte en `[1, 5, 0]`.
      * `[2, 4]` se convierte en `[2, 4, 0]`.

3.  **Tensores**: `torch.tensor()` convierte las listas de Python en **tensores de PyTorch**. Los tensores son la estructura de datos fundamental que PyTorch utiliza para realizar cálculos eficientes en CPUs y GPUs.

-----

### \#\# 2. Definición del Modelo (SentimentRNN) 🧠

Aquí se define la arquitectura de la red neuronal usando una clase que hereda de `nn.Module`. El modelo tiene tres capas principales.

```python
class SentimentRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super().__init__()
        # 1. Capa de Embedding
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        # 2. Capa LSTM
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        # 3. Capa de Salida (Clasificador)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        output, (h_n, c_n) = self.lstm(embedded)
        out = self.fc(h_n[-1])
        return out
```

#### **Las Capas (en `__init__`)**

1.  **`nn.Embedding`**: Esta capa es crucial. Convierte cada ID de palabra (un número como `5`) en un **vector denso** de un tamaño fijo (`embed_dim`, en este caso 8). Este vector, llamado "embedding", captura el significado semántico de la palabra. En lugar de ser solo un número, la palabra "great" se convierte en un vector como `[0.1, -0.5, 0.8, ...]`. El modelo **aprende** estos vectores durante el entrenamiento.

2.  **`nn.LSTM`**: Es el corazón del modelo. Una LSTM (Long Short-Term Memory) es un tipo de red neuronal recurrente (RNN) diseñada para procesar secuencias. Recorre los vectores de las palabras uno por uno, "recordando" el contexto de la oración. `batch_first=True` es una configuración de conveniencia que indica que nuestros datos tendrán la dimensión del lote primero.

3.  **`nn.Linear`**: Es una capa de clasificación final. Toma la información procesada por la LSTM y la transforma en una salida del tamaño deseado (`output_dim`, que es 2). Producirá una puntuación (o "logit") para cada clase (positiva y negativa).

#### **El Flujo de Datos (en `forward`)**

La función `forward` define cómo los datos pasan a través de las capas:

1.  `embedded = self.embedding(x)`: Los IDs de entrada (`x`) se convierten en vectores de embedding.
2.  `output, (h_n, c_n) = self.lstm(embedded)`: La secuencia de vectores se procesa con la LSTM. La LSTM devuelve dos cosas: `output` (el estado de cada paso en la secuencia) y `(h_n, c_n)` (el último estado oculto y de celda, que actúan como un **resumen de toda la oración**).
3.  `out = self.fc(h_n[-1])`: Usamos solo el **último estado oculto** (`h_n[-1]`) como la representación final de la oración. Este vector-resumen se pasa a la capa lineal para obtener las puntuaciones de clasificación finales.

-----

### \#\# 3. Entrenamiento del Modelo ⚙️

Esta sección configura todo lo necesario para el entrenamiento y ejecuta el bucle de entrenamiento.

```python
# Hiperparámetros
vocab_size = 6
embed_dim = 8
hidden_dim = 16
output_dim = 2

# Instanciación
model = SentimentRNN(vocab_size, embed_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Bucle de entrenamiento
for epoch in range(20):
    # 1. Poner a cero los gradientes
    optimizer.zero_grad()
    # 2. Forward pass: Obtener predicciones
    y_pred = model(X)
    # 3. Calcular la pérdida
    loss = criterion(y_pred, y)
    # 4. Backward pass: Calcular gradientes
    loss.backward()
    # 5. Actualizar los pesos
    optimizer.step()
```

1.  **Hiperparámetros**: Son las "perillas" que ajustamos para configurar el modelo, como el tamaño del vocabulario, la dimensión de los embeddings, etc. `output_dim = 2` porque tenemos dos clases: positiva (1) y negativa (0).

2.  **Función de Pérdida (`criterion`)**: `nn.CrossEntropyLoss` es una función muy común para problemas de clasificación. Mide qué tan equivocadas están las predicciones del modelo en comparación con las etiquetas verdaderas (`y`).

3.  **Optimizador (`optimizer`)**: `optim.Adam` es el algoritmo que ajusta los pesos (parámetros) del modelo para minimizar la pérdida. Básicamente, mira el error y dice "vamos a mover los pesos en esta dirección para equivocarnos un poco menos la próxima vez".

4.  **Bucle de Entrenamiento**: Es el proceso iterativo donde el modelo aprende. En cada `epoch` (una pasada completa por todos los datos):

      * `optimizer.zero_grad()`: Limpia los cálculos de la iteración anterior.
      * `y_pred = model(X)`: El modelo hace una predicción (**Forward Pass**).
      * `loss = criterion(...)`: Se calcula el error.
      * `loss.backward()`: Se calculan los gradientes, que indican cómo debe cambiar cada peso para reducir el error (**Backward Pass**).
      * `optimizer.step()`: El optimizador actualiza los pesos del modelo basándose en los gradientes.

-----

### \#\# 4. Prueba con una Nueva Frase 🚀

Después de que el modelo ha sido entrenado, podemos usarlo para hacer predicciones sobre datos que nunca ha visto.

```python
test = torch.tensor([[1, 3, 5]])  # "good movie great"
pred = model(test)
print("Predicción:", torch.argmax(pred, dim=1).item())
```

1.  Se crea un tensor para la frase de prueba `"good movie great"`.
2.  Se pasa por el modelo ya entrenado para obtener las puntuaciones `pred`. El resultado será un tensor con dos valores, por ejemplo `[-2.1, 3.5]`.
3.  `torch.argmax(pred, dim=1)` encuentra el **índice** del valor más alto. En el ejemplo `[-2.1, 3.5]`, el valor más alto está en el índice `1`.
4.  Como definimos nuestras etiquetas (`labels`) con `1` para "positivo", un resultado de `1` significa que el modelo clasificó la frase como **positiva**. `.item()` simplemente extrae el número del tensor.

## Clasificación de Perros y Gatos

## 1. Cargar dataset público (HuggingFace)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchvision.models import resnet18, ResNet18_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

In [ ]:
dataset = load_dataset("microsoft/cats_vs_dogs")

# Dividir en 80% train / 20% test
dataset = dataset["train"].train_test_split(test_size=0.2)

print(dataset)

## 2. Transformaciones (Data Augmentation + Normalización)

In [ ]:
transform_train = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.Lambda(lambda img: img.convert("RGB")),  # ⚠️ convertir a RGB
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

transform_test = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.Lambda(lambda img: img.convert("RGB")),  # ⚠️ convertir a RGB
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

## 3. Adaptador HuggingFace → PyTorch Dataset

In [ ]:
class CatsDogsDataset(Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.data = hf_dataset
        self.transform = transform
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        img = self.data[idx]["image"]
        label = self.data[idx]["labels"]   # ⚠️ usar 'labels' en lugar de 'file'

        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.long)

train_ds = CatsDogsDataset(dataset["train"], transform=transform_train)
test_ds  = CatsDogsDataset(dataset["test"], transform=transform_test)

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
test_dl  = DataLoader(test_ds, batch_size=32)

## 4. Definir modelo (ResNet18 con pesos de ImageNet)

In [ ]:
weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)

# Congelar todas las capas excepto las últimas
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Reemplazar la última capa para 2 clases
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

## 5. Entrenamiento con Early Stopping

In [ ]:
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

#n_epochs = 20
n_epochs = 1
patience = 3
best_acc = 0
epochs_no_improve = 0

for epoch in range(n_epochs):
    # --- Entrenamiento ---
    model.train()
    running_loss = 0.0
    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)
        preds = model(xb)
        loss = loss_fn(preds, yb)

        opt.zero_grad()
        loss.backward()
        opt.step()
        running_loss += loss.item()

    # --- Evaluación ---
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for xb, yb in test_dl:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb).argmax(dim=1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)

    acc = correct / total
    print(f"Epoch {epoch+1}, Loss={running_loss/len(train_dl):.4f}, Val Acc={acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        epochs_no_improve = 0
        torch.save(model.state_dict(), "best_catsdogs.pth")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping activado")
            break

print("Mejor accuracy alcanzado:", best_acc)

## 6. Visualización de predicciones

In [ ]:

images, labels = next(iter(test_dl))
images, labels = images.to(device), labels.to(device)
preds = model(images).argmax(dim=1)

plt.figure(figsize=(12,6))
for i in range(8):
    plt.subplot(2,4,i+1)
    img = images[i].cpu().permute(1,2,0).numpy()
    img = (img * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]).clip(0,1)  # desnormalizar
    plt.imshow(img)
    plt.title(f"Real: {labels[i].item()}, Pred: {preds[i].item()}")
    plt.axis("off")
plt.show()

## Preguntas de Discusión

1. ¿Qué ventajas ofrece usar un modelo preentrenado (transfer learning) frente a entrenar desde cero?
2. ¿Por qué es útil congelar capas en el inicio del entrenamiento y ajustar solo la última capa?
3. ¿Cómo ayuda el *data augmentation* a mejorar la capacidad de generalización del modelo?
4. ¿Qué diferencias observas en el rendimiento al descongelar más capas para el fine-tuning?
5. ¿Qué rol cumple la normalización con los valores de ImageNet en la estabilidad del entrenamiento?
6. ¿Cómo decide el early stopping cuándo detener el entrenamiento y por qué es importante?
7. ¿Qué métricas adicionales (además de accuracy) podrían ser útiles en este problema?
8. ¿Cómo se podría mejorar aún más el modelo si se dispusiera de más recursos computacionales?

## 💡 Preguntas de Discusión (desarrolladas)

1. **¿Qué ventajas ofrece usar un modelo preentrenado (transfer learning) frente a entrenar desde cero?**

   * Entrenar desde cero requiere grandes cantidades de datos y mucho tiempo de cómputo.
   * Los modelos preentrenados en ImageNet ya han aprendido características generales (bordes, texturas, formas), que son útiles para muchos problemas de visión.
   * Con *transfer learning*, solo se adapta la parte final de la red al nuevo conjunto de clases, logrando **mejor rendimiento con menos datos y menos tiempo de entrenamiento**.

---

2. **¿Por qué es útil congelar capas en el inicio del entrenamiento y ajustar solo la última capa?**

   * Las primeras capas de una CNN aprenden características muy generales (líneas, bordes, patrones de color).
   * Si se ajustan todas desde el inicio, se corre el riesgo de *desaprender* esas representaciones útiles.
   * Congelarlas permite entrenar más rápido y reducir el riesgo de sobreajuste, enfocando el aprendizaje solo en la capa de clasificación final.

---

3. **¿Cómo ayuda el *data augmentation* a mejorar la capacidad de generalización del modelo?**

   * Genera versiones modificadas de las imágenes (rotadas, espejadas, con variaciones de color).
   * Esto obliga al modelo a aprender **patrones invariantes** a pequeñas transformaciones, en lugar de memorizar ejemplos concretos.
   * Mejora la robustez y reduce el riesgo de sobreajuste cuando los datasets son pequeños.

---

4. **¿Qué diferencias observas en el rendimiento al descongelar más capas para el fine-tuning?**

   * Congelar casi todo → entrenamiento rápido pero menos capacidad de adaptación al nuevo dominio.
   * Descongelar últimas capas → mejor ajuste al dataset objetivo, a costa de más tiempo de entrenamiento.
   * Descongelar toda la red → mayor capacidad de adaptación, pero mayor riesgo de sobreajuste si el dataset es pequeño.
   * En la práctica, **descongelar gradualmente** (empezando desde las últimas capas) suele dar los mejores resultados.

---

5. **¿Qué rol cumple la normalización con los valores de ImageNet en la estabilidad del entrenamiento?**

   * Los modelos preentrenados esperan entradas con la misma estadística que los datos de ImageNet.
   * Normalizar con `mean=[0.485, 0.456, 0.406]` y `std=[0.229, 0.224, 0.225]` alinea la distribución de píxeles con la que el modelo fue entrenado originalmente.
   * Esto evita desajustes que podrían degradar el rendimiento o dificultar la convergencia.

---

6. **¿Cómo decide el early stopping cuándo detener el entrenamiento y por qué es importante?**

   * Early stopping monitorea una métrica de validación (ej. pérdida o accuracy).
   * Si no mejora después de un número definido de épocas (*patience*), se detiene el entrenamiento.
   * Esto evita que el modelo siga ajustándose al conjunto de entrenamiento mientras empeora en el de validación (sobreajuste).
   * También ahorra tiempo y recursos.

---

7. **¿Qué métricas adicionales (además de accuracy) podrían ser útiles en este problema?**

   * **Precisión (precision):** proporción de predicciones positivas correctas (útil si queremos pocas falsas alarmas).
   * **Recall (sensibilidad):** proporción de verdaderos positivos detectados (útil si no queremos dejar escapar casos).
   * **F1-score:** balance entre precisión y recall.
   * **Matriz de confusión:** para ver qué clases se confunden más.
   * **AUC-ROC:** mide la capacidad de distinguir entre clases en distintos umbrales.

---

8. **¿Cómo se podría mejorar aún más el modelo si se dispusiera de más recursos computacionales?**

   * Usar arquitecturas más grandes y potentes (**ResNet50, EfficientNet, Vision Transformers**).
   * Entrenar durante más épocas con estrategias de regularización (dropout, weight decay).
   * Aumentar la resolución de entrada (224×224 → 384×384).
   * Usar *ensembles* de varios modelos para combinar predicciones.
   * Aplicar *semi-supervised learning* o *self-supervised pretraining* para aprovechar datos no etiquetados.

